In [2]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [3]:
spark.version

'3.5.2'

In [4]:
from pyspark.sql.functions import col, explode

# 1. Cities table

In [64]:
df = spark.read.json(r"C:\Users\karee\Downloads\file_0_20250320_112647.json")

In [65]:
df.show()

+--------------------+-------------------+
|                data|         pagination|
+--------------------+-------------------+
|[{1, Anaa, PF, NU...|{100, 100, 0, 9374}|
+--------------------+-------------------+



In [66]:
df2 = df.withColumn("explodedarray", explode(col("data")))

In [67]:
df2.show()

+--------------------+-------------------+--------------------+
|                data|         pagination|       explodedarray|
+--------------------+-------------------+--------------------+
|[{1, Anaa, PF, NU...|{100, 100, 0, 9374}|{1, Anaa, PF, NUL...|
|[{1, Anaa, PF, NU...|{100, 100, 0, 9374}|{2, Arrabury, AU,...|
|[{1, Anaa, PF, NU...|{100, 100, 0, 9374}|{3, El Arish, EG,...|
|[{1, Anaa, PF, NU...|{100, 100, 0, 9374}|{4, Annaba, DZ, 2...|
|[{1, Anaa, PF, NU...|{100, 100, 0, 9374}|{5, Apalachicola,...|
|[{1, Anaa, PF, NU...|{100, 100, 0, 9374}|{6, Arapoti, BR, ...|
|[{1, Anaa, PF, NU...|{100, 100, 0, 9374}|{7, Aachen, DE, 3...|
|[{1, Anaa, PF, NU...|{100, 100, 0, 9374}|{8, Arraias, BR, ...|
|[{1, Anaa, PF, NU...|{100, 100, 0, 9374}|{9, Awaradam, SR,...|
|[{1, Anaa, PF, NU...|{100, 100, 0, 9374}|{10, Aranuka, KI,...|
|[{1, Anaa, PF, NU...|{100, 100, 0, 9374}|{11, Aalborg, DK,...|
|[{1, Anaa, PF, NU...|{100, 100, 0, 9374}|{12, Mala Mala, Z...|
|[{1, Anaa, PF, NU...|{100, 100, 0, 9374

In [68]:
df2.createOrReplaceTempView("table")

In [77]:
df3 = spark.sql(
    """
        SELECT DISTINCT
            explodedarray.*
        FROM table
        WHERE explodedarray.country_iso2 IS NULL
    """
).show() 

+-------+---------+------------+----------+---+---------+---+--------+---------+--------+
|city_id|city_name|country_iso2|geoname_id|gmt|iata_code| id|latitude|longitude|timezone|
+-------+---------+------------+----------+---+---------+---+--------+---------+--------+
+-------+---------+------------+----------+---+---------+---+--------+---------+--------+



In [123]:
df3 = spark.sql(
    """ 
    SELECT 
        COALESCE(explodedarray.city_id, 0) as city_id,
        COALESCE(explodedarray.city_name, 'Unknown') as city_name,

        COALESCE(explodedarray.iata_code, 'N/A') as iata_code,

        COALESCE(explodedarray.latitude, 0) as latitude,
        COALESCE(explodedarray.longitude, 0) as longitude,
        
        COALESCE(explodedarray.timezone, 'N/A') as timezone,
        COALESCE(explodedarray.country_iso2, 'N/A') as country_iso2
    FROM table   
    """
).show()

+-------+------------+---------+----------+----------+-------------------+------------+
|city_id|   city_name|iata_code|  latitude| longitude|           timezone|country_iso2|
+-------+------------+---------+----------+----------+-------------------+------------+
|      1|        Anaa|      AAA|    -17.05|-145.41667|     Pacific/Tahiti|          PF|
|      2|    Arrabury|      AAB|     -26.7| 141.04167| Australia/Brisbane|          AU|
|      3|    El Arish|      AAC| 31.133333|     33.75|       Africa/Cairo|          EG|
|      4|      Annaba|      AAE| 36.821392|  7.811857|     Africa/Algiers|          DZ|
|      5|Apalachicola|      AAF| 29.733334| -84.98333|    America/Chicago|          US|
|      6|     Arapoti|      AAG|-24.103611|    -49.79|  America/Sao_Paulo|          BR|
|      7|      Aachen|      AAH| 50.775438|   6.08151|      Europe/Berlin|          DE|
|      8|     Arraias|      AAI|-12.916667|-46.933334|  America/Araguaina|          BR|
|      9|    Awaradam|      AAJ|

# 2. Countries Table


In [148]:
df = spark.read.json(r"C:\Users\karee\Downloads\file_101_20250320_114525.json")

In [149]:
df.show()

+--------------------+--------------------+
|                data|          pagination|
+--------------------+--------------------+
|[{Budapest, EU, 1...|{100, 100, 101, 252}|
+--------------------+--------------------+



In [150]:
df = df.withColumn("explodedarray", explode(col('data')))

In [151]:
df.createOrReplaceTempView("table")

In [154]:
df3 = spark.sql(
    """ 
    SELECT 
         explodedarray.*
    FROM table   
    WHERE 
        explodedarray.country_iso3 IS NULL
    """
).show()

+-------+---------+----------+------------+------------+-------------------+------------+-------------+-------------+---------+---+------------+----------+
|capital|continent|country_id|country_iso2|country_iso3|country_iso_numeric|country_name|currency_code|currency_name|fips_code| id|phone_prefix|population|
+-------+---------+----------+------------+------------+-------------------+------------+-------------+-------------+---------+---+------------+----------+
+-------+---------+----------+------------+------------+-------------------+------------+-------------+-------------+---------+---+------------+----------+



In [ ]:
df2 = spark.sql(
    """ 
    SELECT 
        COALESCE(explodedarray.country_name, 'Unknown') as country_name,
        COALESCE(explodedarray.country_iso2, 'N/A') as country_iso2,
        COALESCE(explodedarray.capital, 'Unknown') as capital,
        COALESCE(explodedarray.continent, 'Unknown') as continent,
        COALESCE(explodedarray.currency_name, 'Unknown') as currency_name,
        COALESCE(explodedarray.population, 0) as population,
        COALESCE(explodedarray.phone_prefix, "Unknown") as phone_prefix
    FROM table   
    """
).show()

+----------+--------------------+------------+--------------------+---------+-------------+----------+------------+
|country_id|        country_name|country_iso2|             capital|continent|currency_name|population|phone_prefix|
+----------+--------------------+------------+--------------------+---------+-------------+----------+------------+
|       102|             Hungary|          HU|            Budapest|       EU|       Forint|   9982000|          36|
|       103|           Indonesia|          ID|             Jakarta|       AS|       Rupiah| 242968342|          62|
|       104|             Ireland|          IE|              Dublin|       EU|         Euro|   4622917|         353|
|       105|              Israel|          IL|           Jerusalem|       AS|       Shekel|   7353985|         972|
|       106|         Isle of Man|          IM|Douglas, Isle of Man|       EU|        Pound|     75049|    +44-1624|
|       107|               India|          IN|           New Delhi|     

# 3. Airplanes Table:

In [196]:
df = spark.read.json(r"C:\Users\karee\Downloads\file_1010_20250320_124911.json")

In [197]:
df.show()

+--------------------+--------------------+
|                data|          pagination|
+--------------------+--------------------+
|[{6R, NULL, 1011,...|{100, 100, 1010, ...|
+--------------------+--------------------+



In [198]:
df = df.withColumn("explodedarray", explode(col('data')))   

In [199]:
df.show()

+--------------------+--------------------+--------------------+
|                data|          pagination|       explodedarray|
+--------------------+--------------------+--------------------+
|[{6R, NULL, 1011,...|{100, 100, 1010, ...|{6R, NULL, 1011, ...|
|[{6R, NULL, 1011,...|{100, 100, 1010, ...|{6R, NULL, 1012, ...|
|[{6R, NULL, 1011,...|{100, 100, 1010, ...|{6R, NULL, 1013, ...|
|[{6R, NULL, 1011,...|{100, 100, 1010, ...|{6R, NULL, 1014, ...|
|[{6R, NULL, 1011,...|{100, 100, 1010, ...|{6R, NULL, 1015, ...|
|[{6R, NULL, 1011,...|{100, 100, 1010, ...|{7C, NULL, 1016, ...|
|[{6R, NULL, 1011,...|{100, 100, 1010, ...|{7C, NULL, 1017, ...|
|[{6R, NULL, 1011,...|{100, 100, 1010, ...|{7C, NULL, 1018, ...|
|[{6R, NULL, 1011,...|{100, 100, 1010, ...|{7C, NULL, 1019, ...|
|[{6R, NULL, 1011,...|{100, 100, 1010, ...|{7C, NULL, 1020, ...|
|[{6R, NULL, 1011,...|{100, 100, 1010, ...|{7C, NULL, 1021, ...|
|[{6R, NULL, 1011,...|{100, 100, 1010, ...|{7C, NULL, 1022, ...|
|[{6R, NULL, 1011,...|{10

In [200]:
df.createOrReplaceTempView("table")

In [201]:
df3 = spark.sql(
    """ 
        SELECT 
            explodedarray.*
        FROM table   
        WHERE 
            explodedarray.registration_number IS NULL
    """
).show()

+-----------------+-----------------+-----------+-------------------+-------------+-------------+------------+-----------------+--------------+---------------+---------+-------------+---+-----------+----------+----------+---------+-----------+-----------+------------+------------+---------------+-----------------+-------------------+------------+------------------------+
|airline_iata_code|airline_icao_code|airplane_id|construction_number|delivery_date|engines_count|engines_type|first_flight_date|iata_code_long|iata_code_short|iata_type|icao_code_hex| id|line_number|model_code|model_name|plane_age|plane_class|plane_owner|plane_series|plane_status|production_line|registration_date|registration_number|rollout_date|test_registration_number|
+-----------------+-----------------+-----------+-------------------+-------------+-------------+------------+-----------------+--------------+---------------+---------+-------------+---+-----------+----------+----------+---------+-----------+---------

In [154]:
df2 = spark.sql(
    """ 
        SELECT 
            COALESCE(explodedarray.registration_number, 'Unknown') as registration_number,
            COALESCE(explodedarray.model_name, 'Unknown') as model_name,
            COALESCE(explodedarray.model_code, 'Unknown') as model_code,
            COALESCE(explodedarray.plane_owner, 'Unknown') as plane_owner,


            COALESCE(explodedarray.construction_number, 0) as construction_number,
            COALESCE(explodedarray.production_line, 'Unknown') as production_line,
            COALESCE(explodedarray.iata_type, 'Unknown') as iata_type,

            COALESCE(explodedarray.first_flight_date, CAST('0001-01-01T00:00:00.000Z' AS TIMESTAMP)) as first_flight_date,
            COALESCE(explodedarray.delivery_date, CAST('0001-01-01T00:00:00.000Z' AS TIMESTAMP)) as delivery_date,

            COALESCE(explodedarray.plane_series, 'Unknown') as plane_series,
            COALESCE(explodedarray.engines_count, 0) as engines_count,
            COALESCE(explodedarray.engines_type, 'Unknown') as engines_type,
            COALESCE(explodedarray.plane_age, 0) as plane_age,
            COALESCE(explodedarray.plane_status, 'Unknown') as plane_status

        FROM table   
    """
).show()

+-------------------+----------+-------------+--------------------+-------------------+----------------+---------+--------------------+--------------------+------------+-------------+------------+---------+------------+
|registration_number|model_name|   model_code|         plane_owner|construction_number| production_line|iata_type|   first_flight_date|       delivery_date|plane_series|engines_count|engines_type|plane_age|plane_status|
+-------------------+----------+-------------+--------------------+-------------------+----------------+---------+--------------------+--------------------+------------+-------------+------------+---------+------------+
|             XA-FPP|      A300|A300B4-203(F)|Wells Fargo Bank ...|                227|Airbus A300/A310|   A300B4|1983-02-28T23:00:...|1982-12-20T23:00:...|     B4-203F|            2|         JET|       34|      active|
|             XA-LRC|       767|B767-241(ERF)|Wells Fargo Bank ...|              23802|      Boeing 767| B767-200|1987-0

# 4. Airports Table

In [202]:
df = spark.read.json(r"C:\Users\karee\Downloads\file_1111_20250320_121457.json")

In [203]:
df.show()

+--------------------+--------------------+
|                data|          pagination|
+--------------------+--------------------+
|[{1465, Cotulla, ...|{100, 100, 1111, ...|
+--------------------+--------------------+



In [204]:
df= df.withColumn("explodedarray", explode(col('data')))
df.show()

+--------------------+--------------------+--------------------+
|                data|          pagination|       explodedarray|
+--------------------+--------------------+--------------------+
|[{1465, Cotulla, ...|{100, 100, 1111, ...|{1465, Cotulla, C...|
|[{1465, Cotulla, ...|{100, 100, 1111, ...|{1466, Columbia R...|
|[{1465, Cotulla, ...|{100, 100, 1111, ...|{1467, Covilha, C...|
|[{1465, Cotulla, ...|{100, 100, 1111, ...|{1472, A. Tubman,...|
|[{1465, Cotulla, ...|{100, 100, 1111, ...|{1473, Capurgana,...|
|[{1465, Cotulla, ...|{100, 100, 1111, ...|{1474, Chapelco, ...|
|[{1465, Cotulla, ...|{100, 100, 1111, ...|{1475, Coober Ped...|
|[{1465, Cotulla, ...|{100, 100, 1111, ...|{1476, Campeche I...|
|[{1465, Cotulla, ...|{100, 100, 1111, ...|{1477, Cepu, CPF,...|
|[{1465, Cotulla, ...|{100, 100, 1111, ...|{1479, Kastrup, C...|
|[{1465, Cotulla, ...|{100, 100, 1111, ...|{1481, Chaparral,...|
|[{1465, Cotulla, ...|{100, 100, 1111, ...|{1482, Compton, C...|
|[{1465, Cotulla, ...|{10

In [205]:
df.createOrReplaceTempView("table")

In [207]:
df3 = spark.sql(
    """ 
    SELECT 
        explodedarray.*
    FROM table
    WHERE explodedarray.iata_code IS NULL   
    """
).show()

+----------+------------+--------------+------------+------------+----------+---+---------+---------+---+--------+---------+------------+--------+
|airport_id|airport_name|city_iata_code|country_iso2|country_name|geoname_id|gmt|iata_code|icao_code| id|latitude|longitude|phone_number|timezone|
+----------+------------+--------------+------------+------------+----------+---+---------+---------+---+--------+---------+------------+--------+
+----------+------------+--------------+------------+------------+----------+---+---------+---------+---+--------+---------+------------+--------+



In [30]:
df2 = spark.sql(
    """ 
    SELECT 
        COALESCE(explodedarray.airport_id, 0) as airport_id,
        COALESCE(explodedarray.airport_name, 'Unknown') as airport_name,
        COALESCE(explodedarray.iata_code, 'N/A') as iata_code,
        COALESCE(explodedarray.icao_code, 'N/A') as icao_code,
        COALESCE(explodedarray.city_iata_code, 'N/A') as city_iata_code,
        
        COALESCE(explodedarray.latitude, 0) as latitude,
        COALESCE(explodedarray.longitude, 0) as longitude,
        COALESCE(explodedarray.timezone, 'N/A') as timezone,

        COALESCE(explodedarray.country_iso2, 'N/A') as country_iso2,
        COALESCE(explodedarray.country_name, 'Unknown') as country_name

    FROM table
    WHERE explodedarray.icao_code IS NULL   
    """
).show()

+----------+------------+---------+---------+--------------+--------+---------+--------+------------+------------+
|airport_id|airport_name|iata_code|icao_code|city_iata_code|latitude|longitude|timezone|country_iso2|country_name|
+----------+------------+---------+---------+--------------+--------+---------+--------+------------+------------+
+----------+------------+---------+---------+--------------+--------+---------+--------+------------+------------+



# 5. Airlines Table:

In [188]:
df = spark.read.json(r"C:\Users\karee\Downloads\file_10504_20250320_122519.json")

In [189]:
df.show()

+--------------------+--------------------+
|                data|          pagination|
+--------------------+--------------------+
|[{10506, Planemas...|{100, 100, 10504,...|
+--------------------+--------------------+



In [190]:
df= df.withColumn("explodedarray", explode(col('data')))
df.show()

+--------------------+--------------------+--------------------+
|                data|          pagination|       explodedarray|
+--------------------+--------------------+--------------------+
|[{10506, Planemas...|{100, 100, 10504,...|{10506, Planemast...|
|[{10506, Planemas...|{100, 100, 10504,...|{10507, Perm Moto...|
|[{10506, Planemas...|{100, 100, 10504,...|{10508, Premium A...|
|[{10506, Planemas...|{100, 100, 10504,...|{10509, Polizeihu...|
|[{10506, Planemas...|{100, 100, 10504,...|{10510, Petroleos...|
|[{10506, Planemas...|{100, 100, 10504,...|{10511, Pink Avia...|
|[{10506, Planemas...|{100, 100, 10504,...|{10512, Universal...|
|[{10506, Planemas...|{100, 100, 10504,...|{10513, Pan-Air, ...|
|[{10506, Planemas...|{100, 100, 10504,...|{10514, Prince Av...|
|[{10506, Planemas...|{100, 100, 10504,...|{10515, Pond Air ...|
|[{10506, Planemas...|{100, 100, 10504,...|{10516, Peninter ...|
|[{10506, Planemas...|{100, 100, 10504,...|{10517, PanAfriqi...|
|[{10506, Planemas...|{10

In [191]:
df.createOrReplaceTempView("table")

In [194]:
df2 = spark.sql(
    """ 
    SELECT 
        explodedarray.*

    FROM table
    where explodedarray.icao_code is  null
    """
).show()

+----------+------------+--------+------------+------------+------------+-----------------+----------+--------+---------+----------------------+---------+---+------+----+
|airline_id|airline_name|callsign|country_iso2|country_name|date_founded|fleet_average_age|fleet_size|hub_code|iata_code|iata_prefix_accounting|icao_code| id|status|type|
+----------+------------+--------+------------+------------+------------+-----------------+----------+--------+---------+----------------------+---------+---+------+----+
+----------+------------+--------+------------+------------+------------+-----------------+----------+--------+---------+----------------------+---------+---+------+----+



In [23]:
df2 = spark.sql(
    """ 
    SELECT 
        explodedarray.airline_id,
        COALESCE(explodedarray.airline_name, "Unknown") as airline_name,
        COALESCE(explodedarray.iata_code,"N/A") as iata_code,
        COALESCE(explodedarray.iata_prefix_accounting,"N/A") as iata_prefix_accounting,
        COALESCE(explodedarray.icao_code,"N/A") as icao_code,

        COALESCE(explodedarray.country_iso2,"N/A") as country_iso2,
        COALESCE(explodedarray.country_name,"Unknown") as country_name,
        COALESCE(explodedarray.callsign,"N/A") as callsign,

        COALESCE(explodedarray.fleet_size,0) as fleet_size,
        COALESCE(explodedarray.fleet_average_age,0.0) as fleet_average_age,

        COALESCE(explodedarray.date_founded,0000) as date_founded,

        COALESCE(explodedarray.status,"Unknown") as status,
        COALESCE(explodedarray.type,"Unknown") as type

    FROM table
    where explodedarray.icao_code is  null
    """
).show()

+----------+------------+---------+----------------------+---------+------------+------------+--------+----------+-----------------+------------+------+----+
|airline_id|airline_name|iata_code|iata_prefix_accounting|icao_code|country_iso2|country_name|callsign|fleet_size|fleet_average_age|date_founded|status|type|
+----------+------------+---------+----------------------+---------+------------+------------+--------+----------+-----------------+------------+------+----+
+----------+------------+---------+----------------------+---------+------------+------------+--------+----------+-----------------+------------+------+----+



# 6. Flight Table


In [5]:
df = spark.read.json(r"C:\Users\karee\Downloads\file_2323_20250319_115001.json")

In [6]:
df.show()

+--------------------+--------------------+
|                data|          pagination|
+--------------------+--------------------+
|[{{B738, B738, 4B...|{100, 100, 2323, ...|
+--------------------+--------------------+



In [7]:
df= df.withColumn("explodedarray", explode(col('data')))
df.show()

+--------------------+--------------------+--------------------+
|                data|          pagination|       explodedarray|
+--------------------+--------------------+--------------------+
|[{{B738, B738, 4B...|{100, 100, 2323, ...|{{B738, B738, 4BA...|
|[{{B738, B738, 4B...|{100, 100, 2323, ...|{{A20N, A20N, 4BB...|
|[{{B738, B738, 4B...|{100, 100, 2323, ...|{{A21N, A21N, 4BC...|
|[{{B738, B738, 4B...|{100, 100, 2323, ...|{NULL, {XY, KNE, ...|
|[{{B738, B738, 4B...|{100, 100, 2323, ...|{{A21N, A21N, 4BC...|
|[{{B738, B738, 4B...|{100, 100, 2323, ...|{NULL, {PX, ANG, ...|
|[{{B738, B738, 4B...|{100, 100, 2323, ...|{NULL, {CG, TOK, ...|
|[{{B738, B738, 4B...|{100, 100, 2323, ...|{NULL, {UA, UAL, ...|
|[{{B738, B738, 4B...|{100, 100, 2323, ...|{NULL, {NULL, NUL...|
|[{{B738, B738, 4B...|{100, 100, 2323, ...|{NULL, {NH, ANA, ...|
|[{{B738, B738, 4B...|{100, 100, 2323, ...|{{A21N, A21N, 781...|
|[{{B738, B738, 4B...|{100, 100, 2323, ...|{{B738, B738, 780...|
|[{{B738, B738, 4B...|{10

In [8]:
df.createOrReplaceTempView("table")

In [54]:
df3= spark.sql(
    """
    SELECT DISTINCT 
        explodedarray.arrival.icao
    FROM table
    where explodedarray.flight.iata is null
    """).show()

+----+
|icao|
+----+
|YRTI|
+----+



In [ ]:
df2 = spark.sql(
    """
    SELECT 
    -- Flight attributes
    COALESCE(explodedarray.flight.iata, 'Unknown') AS flight_iata, --pk
    COALESCE(explodedarray.flight.icao, 'Unknown') AS flight_icao,
    COALESCE(explodedarray.flight.number, 0000) AS flight_number,
    COALESCE(explodedarray.flight_date, '0001-01-01') AS flight_date, -- Default date
    COALESCE(explodedarray.flight_status, 'Unknown') AS flight_status,

    -- Departure attributes
    COALESCE(explodedarray.departure.icao, 'Unknown') AS departure_icao, -- fk
    COALESCE(explodedarray.departure.terminal, 'N/A') AS departure_terminal,
    COALESCE(explodedarray.departure.gate, 'N/A') AS departure_gate,
    COALESCE(explodedarray.departure.scheduled, CAST('0001-01-01T00:00:00.000Z' AS TIMESTAMP)) AS departure_scheduled,
    COALESCE(explodedarray.departure.actual, CAST('0001-01-01T00:00:00.000Z' AS TIMESTAMP)) AS departure_actual,

    -- Arrival attributes
    COALESCE(explodedarray.arrival.icao, 'Unknown') AS arrival_icao, -- fk
    COALESCE(explodedarray.arrival.terminal, 'N/A') AS arrival_terminal,
    COALESCE(explodedarray.arrival.gate, 'N/A') AS arrival_gate,
    COALESCE(explodedarray.arrival.baggage, 'N/A') AS arrival_baggage,
    COALESCE(explodedarray.arrival.scheduled, CAST('0001-01-01T00:00:00.000Z' AS TIMESTAMP)) AS arrival_scheduled,
    COALESCE(explodedarray.arrival.actual, CAST('0001-01-01T00:00:00.000Z' AS TIMESTAMP)) AS arrival_actual,

    -- Airline attributes
    COALESCE(explodedarray.airline.icao, 'Unknown') AS airline_icao, -- fk

    -- Aircraft attributes
    COALESCE(explodedarray.aircraft.registration, 'Unknown') AS aircraft_registration, -- fk
    COALESCE(explodedarray.aircraft.iata, 'Unknown') AS aircraft_iata,

    -- Live attributes
    COALESCE(explodedarray.live.is_ground, False) AS live_is_ground -- Default: False

FROM table
WHERE explodedarray.flight.iata IS NOT NULL
"""
).show()

SyntaxError: invalid syntax. Perhaps you forgot a comma? (2557941959.py, line 2)